# `mle-logging`: A Lightweight Logger for ML Experiments
### Author: [@RobertTLange](https://twitter.com/RobertTLange)

There are few things that bring me more joy, than automating and refactoring code, which I use on a daily basis. It feels empowering (when done right) and can lead to serious time savings. One key ingredient to my workflow is the logging of neural network training trajectories and their diagnostics (predictions, checkpoints, etc.). At the beginning I often only saved lists of training/validation/test losses to `.csv` files or used simple tensorboard template code. Maybe with a time string in front of the filename. But this soon became annoying as I wanted to compare runs across hyperparameters configurations and random seeds. Which brought me to a fundamental question: **What makes a good logger for Machine Learning experiments?** After looking at some of my previous projects, I drafted a list of desired properties:

1. **Generality**: The logger should support different types of experiments and provide the functionality to store/retrieve their key diagnostics. This includes time-series statistics such as losses or predictive accuracy, network checkpoints of various flavors, generated figures as well as any other objects one might want to save over the course of training.
2. **Reproducibility**: The logger should provide all the necessary information in order to reproduce the statistics stored with it. This includes the hyperparameter configuration and random seed of the trained pipeline.
3. **Integratability**: Given that we may want to search over hyperparameters and rerun experiments over multiple random seeds, the logger has to be able to easily combine these different sub-logs.
4. **Usability**: The API has to pleasant and intuitive to use. Furthermore, the log aggregation should require a minimal amount of information regarding the file locations. 

Based on these considerations and some iterations I came up with the following design: 

![](https://github.com/RobertTLange/mle-logging/blob/main/docs/mle_logger_structure.png)

Let's now walk through the individual logging steps, how to reload the log and how to aggregate multiple logs:

In [5]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

#!pip install -q mle-logging
from mle_logging import MLELogger

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Storing Basic Logging Results (Stats, Checkpoints, Plots)

We start by creating an instance of our logger. The logger takes a set of minimal inputs: `time_to_track`, `what_to_track`. They are both lists of strings which provide the time and statistic variable names, we want to log over time. There are also a couple more optional basic ingredients:

- `time_to_print`, `what_to_print` and `print_every_k_updates`: By providing these arguments, you tell the logger what to print out onto the console after a set of `k` updates.
- `experiment_dir`: The base directory in which you want to store all your results in.
- `config_fname`: The filename of hyperparameter `.json` configuration for this experiment that will be copied into the `experiment_dir`. 
- `use_tboard`: A boolean which states whether to log statistics also to a TensorBoard.
- `model_type`: A str (`jax`, `torch`, `tensorflow`, `sklearn`, `numpy`) specifying the model format to save.
- `overwrite_experiment_dir`: Whether to overwrite/replace a previously stored log.

In [6]:
# Instantiate logging to experiment_dir
log = MLELogger(time_to_track=['num_updates', 'num_epochs'],
                what_to_track=['train_loss', 'test_loss'],
                time_to_print=['num_updates', 'num_epochs'],
                what_to_print=['train_loss', 'test_loss'],
                experiment_dir="experiment_dir/",
                config_fname=None,
                use_tboard=True,
                model_type='torch',
                print_every_k_updates=1,
                overwrite_experiment_dir=True)

We can then simply log some a time-series "tick"/timestamp by providing the key, value pairs to `log.update()`. The log will print the last provided statistics using `rich` formatting:

In [7]:
# Save some time series statistics
time_tic = {'num_updates': 10,
            'num_epochs': 1}
stats_tic = {'train_loss': 0.1234,
             'test_loss': 0.1235}

# Update the log with collected data & save it to .hdf5
log.update(time_tic, stats_tic)
log.save()

OSError: Can't write data (no appropriate function for conversion path)

Furthermore, we can save the most current checkpoint via `log.save_model()`. Here is an example for how to store a torch model checkpoint:

In [ ]:
# Save a model (torch, sklearn, jax, numpy)
import torch.nn as nn

class DummyModel(nn.Module):
    def __init__(self):
        super(DummyModel, self).__init__()
        self.fc1 = nn.Linear(28*28, 300)
        self.fc2 = nn.Linear(300, 100)
        self.fc3 = nn.Linear(100, 10)

    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x
    
model = DummyModel()
log.save_model(model)

If you would like to save a figure that was generated during your training loop, this can be done via `log.save_plot()`. More general objects can be saved as `.pkl` via `log.save_extra`. The log will keep a counter of how many figures or objects were previously saved. If you do not provide an explicit path to the function calls, this counter will be used to archive the files chronologically.

In [ ]:
# Save a matplotlib figure as .png
import numpy as np
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
ax.plot(np.random.normal(0, 1, 20))
log.save_plot(fig)
# You can also explicity give a name to the file
# log.save_plot(fig, "some_figure_path.png")

# You can also save (somewhat) arbitrary objects .pkl
some_dict = {"hi" : "there"}
log.save_extra(some_dict)

# You can also explicity give a name to the file
# log.save_extra(some_dict, "some_object_path.pkl")

And obviously you do not need to go through these steps individually, but can also save statistics, model checkpoint, a plot and extra content all in one go: 

In [ ]:
# Or do everything in one go
log.update(time_tic, stats_tic, model, fig, some_dict, save=True)

# Reloading for Post-Processing

Reload can be done via `load_log(<experiment_dir>)`. The reloaded log is a `dotmap` dictionary, which has three subkeys: 

1. `meta`: The meta-information of the experiment. This includes random seed, configuration path, figure paths, experiment directory, etc..
2. `time`: The time-series for the `time` variables.
3. `stats`: The time-series for the `stats` variables.

The individual data can be accessed via indexing with `.` - have a look:

In [ ]:
from mle_logging import load_log

log = load_log("experiment_dir/")

In [ ]:
log.meta.keys()

In [ ]:
log.meta.model_ckpt[0].decode()

In [ ]:
log.stats.keys()

In [ ]:
log.stats.test_loss

In [ ]:
log.time.keys()

In [ ]:
log.time.num_updates

# Log Different Random Seeds for Same Configuration

If you provide a `.json` file path and a seed_id, the log will be created in a sub-directory. Furthermore, the `.json` file will be copied for reproducibility. Multiple simultanous runs (different seeds) can now log to the same directory. Everything else remains the same.

In [ ]:
# Instantiate logging to experiment_dir for two random seeds
log_seed_1 = MLELogger(time_to_track=['num_updates', 'num_epochs'],
                       what_to_track=['train_loss', 'test_loss'],
                       experiment_dir="multi_seed_dir/",
                       config_fname="config_1.json",     # Provide path to .json config
                       seed_id="seed_1")                 # Provide some seed identifier (str)   

log_seed_2 = MLELogger(time_to_track=['num_updates', 'num_epochs'],
                       what_to_track=['train_loss', 'test_loss'],
                       experiment_dir="multi_seed_dir/",
                       config_fname="config_1.json",     # Provide path to .json config
                       seed_id="seed_2")                 # Provide some seed identifier (str)   

# Save some time series statistics
time_tic = {'num_updates': 10,
            'num_epochs': 1}
stats_tic = {'train_loss': 0.1234,
             'test_loss': 0.1235}

# Update the log with collected data & save them to .hdf5
log_seed_1.update(time_tic, stats_tic, save=True)
log_seed_2.update(time_tic, stats_tic, save=True)

We can then use `merge_seed_logs` in order to combine both `.hdf5` log files (for the different seeds) into a single file stored in `merged_path`. The `load_log` function afterwards will load this seed-merged log and the first level of the `dotmap` dictionary will give you an overview of the different random seeds: 

In [ ]:
import os, datetime
from mle_logging import merge_seed_logs, load_log

timestr = datetime.datetime.today().strftime("%Y-%m-%d")[2:]
experiment_dir = f"multi_seed_dir/{timestr}_config_1/"
merged_path = os.path.join(experiment_dir, "logs", "seed_aggregated.hdf5")

# Merge different random seeds into one .hdf5 file
merge_seed_logs(merged_path, experiment_dir)

# Load the merged log - Individual seeds can be accessed via log.seed_1, etc.
log = load_log(experiment_dir)
log.eval_ids, log.seed_1.stats.train_loss

You can also directly aggregate these different random seeds by setting `aggregate_seeds=True`. This will compute the mean, standard deviation as well as different percentiles over the random seeds.

In [ ]:
# Load the merged log and aggregate over random seeds (compute stats)
log = load_log(experiment_dir, aggregate_seeds=True)
log.eval_ids, log.stats.train_loss.keys()

## Log Different Configurations with Different Random Seeds

Next, we can also combine different logs for different hyperparameter configurations and their random seeds. Let's first create two logs for two different configurations:

In [ ]:
# Instantiate logging to experiment_dir for two .json configurations and two seeds
log_c1_s1 = MLELogger(time_to_track=['num_updates', 'num_epochs'],
                      what_to_track=['train_loss', 'test_loss'],
                      experiment_dir="multi_config_dir/",
                      config_fname="config_1.json",     # Provide path to .json config
                      seed_id="seed_1")                 # Provide some seed identifier (str)   

log_c1_s2 = MLELogger(time_to_track=['num_updates', 'num_epochs'],
                      what_to_track=['train_loss', 'test_loss'],
                      experiment_dir="multi_config_dir/",
                      config_fname="config_1.json",     # Provide path to .json config
                      seed_id="seed_2")                 # Provide some seed identifier (str)   

log_c2_s1 = MLELogger(time_to_track=['num_updates', 'num_epochs'],
                      what_to_track=['train_loss', 'test_loss'],
                      experiment_dir="multi_config_dir/",
                      config_fname="config_2.json",     # Provide path to .json config
                      seed_id="seed_1")                 # Provide some seed identifier (str)   

log_c2_s2 = MLELogger(time_to_track=['num_updates', 'num_epochs'],
                      what_to_track=['train_loss', 'test_loss'],
                      experiment_dir="multi_config_dir/",
                      config_fname="config_2.json",     # Provide path to .json config
                      seed_id="seed_2")                 # Provide some seed identifier (str)

# Update the logs with collected data & save them to .hdf5
log_c1_s1.update(time_tic, stats_tic, save=True)
log_c1_s2.update(time_tic, stats_tic, save=True)
log_c2_s1.update(time_tic, stats_tic, save=True)
log_c2_s2.update(time_tic, stats_tic, save=True)

We can now first merge different random seeds for both configurations (again via `merge_seed_logs`) and then afterwards, combine the seed-aggregated logs for the two configurations via `merge_config_logs`:

In [ ]:
# Merge different random seeds for each config into separate .hdf5 file
merge_seed_logs(f"multi_config_dir/{timestr}_config_1/logs/config_1.hdf5", 
                f"multi_config_dir/{timestr}_config_1/")
merge_seed_logs(f"multi_config_dir/{timestr}_config_2/logs/config_2.hdf5", 
                f"multi_config_dir/{timestr}_config_2/")

# Aggregate the different merged configuration .hdf5 files into single meta log
from mle_logging import merge_config_logs
merge_config_logs(experiment_dir="multi_config_dir/",
                  all_run_ids=["config_1", "config_2"])

This meta-log can then be reloaded via `load_meta_log` and specifying its location:

In [ ]:
# Afterwards load in the meta log object
from mle_logging import load_meta_log
meta_log = load_meta_log("multi_config_dir/meta_log.hdf5")
meta_log.eval_ids, meta_log.config_1.stats.test_loss.keys()

Again `load_meta_log` has the option to `aggregate_seeds` or not:

In [ ]:
meta_log = load_meta_log("multi_config_dir/meta_log.hdf5", aggregate_seeds=False)
meta_log.eval_ids

# Logging Every k-th Checkpoint Update

Next up you can choose to not only store the most recent checkpoint, but to also store every k-th one. Simply specify `save_every_k_ckpt` when instantiating the logger and the toolbox will take care of the archiving:

In [ ]:
# Instantiate logging to experiment_dir
log = MLELogger(time_to_track=['num_updates', 'num_epochs'],
                what_to_track=['train_loss', 'test_loss'],
                experiment_dir='every_k_dir/',
                model_type='torch',
                ckpt_time_to_track='num_updates',
                save_every_k_ckpt=2)

In [ ]:
time_tic = {'num_updates': 10, 'num_epochs': 1}
log.update(time_tic, stats_tic, model, save=True)

time_tic = {'num_updates': 20, 'num_epochs': 1}
log.update(time_tic, stats_tic, model, save=True)

time_tic = {'num_updates': 30, 'num_epochs': 1}
log.update(time_tic, stats_tic, model, save=True)

time_tic = {'num_updates': 40, 'num_epochs': 1}
log.update(time_tic, stats_tic, model, save=True)

log.model_log.every_k_ckpt_list, log.model_log.every_k_storage_time

# Logging Top-k Checkpoints Based on Metric

Last but not least we can also choose to keep an archive of the top-k performing networks based on chosen metric. As additional input you have to provide the size of your archive `save_top_k_ckpt`, the metric `top_k_metric_name` and whether to minimize it `top_k_minimize_metric`:

In [ ]:
# Instantiate logging to experiment_dir
log = MLELogger(time_to_track=['num_updates', 'num_epochs'],
                what_to_track=['train_loss', 'test_loss'],
                experiment_dir="top_k_dir/",
                model_type='torch',
                ckpt_time_to_track='num_updates',
                save_top_k_ckpt=2,
                top_k_metric_name="test_loss",
                top_k_minimize_metric=True)

In [ ]:
time_tic = {'num_updates': 10, 'num_epochs': 1}
stats_tic = {'train_loss': 0.1234, 'test_loss': 0.1235}
log.update(time_tic, stats_tic, model, save=True)

time_tic = {'num_updates': 20, 'num_epochs': 1}
stats_tic = {'train_loss': 0.1234, 'test_loss': 0.11}
log.update(time_tic, stats_tic, model, save=True)

time_tic = {'num_updates': 30, 'num_epochs': 1}
stats_tic = {'train_loss': 0.1234, 'test_loss': 0.09}
log.update(time_tic, stats_tic, model, save=True)

time_tic = {'num_updates': 40, 'num_epochs': 1}
stats_tic = {'train_loss': 0.1234, 'test_loss': 0.12}
log.update(time_tic, stats_tic, model, save=True)

log.model_log.top_k_performance, log.model_log.top_k_storage_time

So this is it. Let me know what you think! If you find a bug or are missing your favourite feature, feel free to contact me [@RobertTLange](https://twitter.com/RobertTLange) or create an issue!